<a href="https://colab.research.google.com/github/TD1138/Interactive-Movie-Recommender/blob/main/notebooks/Interactive_Movie_Recommender_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Interactive Movie Recommender using Tensorflow

### Most movie recommender systems I've seen before require a starting point.
### This can be in the form of:
* ###   a reference movie (if you enjoyed X you will enjoy Y or Z)
* ###   a taste profile ('you liked A, B & C so how about D or E')

### I have 2 problems with these kinds of recommender systems:
### If I ask you where you want to go for dinner, it's an open-ended question - one is often  unable to come up with something. However if I ask where you'd rather go from 3 options, that is a much easier question to answer! Once we have a starting point we can start to narrow things down. This is how I'd like a recommender system to work - give me some options and I'll be able to narrow things down!
### The second issue is the need for a starting point - I either need to think of a film that I want any recommendations to be similar to, or I need to have already filled out various films I enjoy. In reality, I sometimes don't know exactly what film I want a film to be similar to, I jsut know what kind of thing I'm after!


### So my idea is to make a recommender that is more interactive than a classic  - taking input from the user.


### For example:
#### - User inputs 'Sci-Fi'
#### * Algorithm outputs 10 'Sci-Fi movies'
#### - User sees that the movies skew quite modern - enters '80s'
#### * Algorithm outputs 10 '80s Sci-Fi movies'
#### - User sees most of these movies feature male leads - adds 'female lead'
#### * Algorithm outputs 10 '80s Sci-Fi movies with Female Leads'

### My idea is to create a neural network with inputs being various features like year, director etc, but also some more meta features, like the tags you see in Netflix's micro-genres - like 'gory' 'visualy striking' etc.
### The output of the neural network would be a softmax with all the films in our dataset. The top 10 probabilities would then be served up as the recommendations.
### It will require a layer of natural language processing to get from the user inputs to the feature space - this will be a second phase of this project!

### But before I can get started with this grand ambition, I need some data! 


# The Data

### GroupLens is a computer science research lab at the University of Minnesota, specializing in recommender systems, among other things.
### They have a dataset which is ideal for my usecase - 'MovieLens 25M'
### Released December 2019, it contains 62k movies, with tags like the ones I was talking about above! See further documentation below:
### https://grouplens.org/datasets/movielens/


### The data is hosted on the GroupLens website, as a zip file.
### First I need to save the ZIP file to the local machine, then use a package to extract the data:

In [1]:
dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-25m.zip'

In [2]:
import os
local_path_zip = os.path.join('..', 'dataset/ml-25m.zip')
local_path = os.path.join('..', 'dataset/ml-25m.zip/ml-25m')

In [3]:
import requests, zipfile, io
r = requests.get(dataset_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(local_path_zip)

####Let's have a look at the files we have downloaded:

In [79]:
import pandas as pd
for file in os.listdir(local_path):
  if file[-4:] == '.csv':
    print(file)
    display(pd.read_csv(local_path+'/'+file, nrows=5))
  else:
    print(file+' is not a csv')
  print('\n')

genome-scores.csv


,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075




links.csv


,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862




ratings.csv


...



genome-tags.csv


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s




movies.csv


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy




README.txt is not a csv




### Let's take a closer look at 'tags.csv' - this file contains every single tag action done by users:

In [191]:
tags = pd.read_csv(local_path+'/tags.csv')
print('There are {} unique tags in the dataset'.format(len(tags['tag'].unique())))
tag_counts = tags[['tag', 'movieId']].groupby('tag').count().sort_values('tag').reset_index()
tag_counts.head(50)

There are 73051 unique tags in the dataset


,tag,movieId
0,Alexander Skarsgård,1
1,Difficult to find it,1
2,Filmes Antigos,2
3,Filmes Antigos,2
4,Kartik Aaryan,1
5,Kriti Sanon,1
6,Laurel Canyon,1
7,Luis Brandoni,1
8,Masami Nagasawa,1
9,O'Shea Jackson Jr.,1


### As we can see, there's a lot of standard text cleansing processes we can do to make things a bit clearer, including stripping any spaces, getting rid of odd characters etc - let's do this now:

In [210]:
def string_cleanse(string):
  string_clean = str(string).strip().lower().replace('"','').replace('#','').replace('\'','').replace('(','').replace(')','').replace('*','').replace('-','')
  return string_clean
tags['tag'] = tags['tag'].apply(string_cleanse)
clean_tag_counts = tags[['tag', 'movieId']].groupby('tag').count().sort_values('tag').reset_index()
clean_tag_counts.head(50)

,tag,movieId
0,,11
1,!950s superman tv show,1
2,&suspense,1
3,+++++++++++++++,1
4,",music business",1
5,.,1
6,...and this film has got nothing to with the a...,1
7,...livin in an amish paradise,1
8,...that its finally over,1
9,...to pay me to watch this again.,1


Next let's start to get some counts of how often these tags appear in the dataset.

Each row of the dataset is a tag applied by a user to a film - let's filter to a single, popular film, to get a feel for what this looks like - I have chosen 2001: A Space Odyssey as it is one of my favourites and is also quite a popular film. This film has id '924'

In [211]:
chosen_film_id = 924
chosen_film_tags = tags[tags['movieId'] == chosen_film_id]
print('Film ID {} has {} total tags, of which {} are unique, from {} different users'.format(chosen_film_id, chosen_film_tags['tag'].count(), chosen_film_tags['tag'].nunique(), chosen_film_tags['userId'].nunique()))

Film ID 924 has 1749 total tags, of which 185 are unique, from 323 different users


### Let's look at the top 50 most used tags:
### This however desn't account for the fact that certain tags might be repeatedly used for certain films - e.g. the tag 'Harry Potter' might be repeatedly used across all the Harry Potter franchise - but isn't all that useful for our recommender.

In [212]:
tags[['tag', 'movieId']].groupby('tag').count().sort_values('movieId', ascending=False).reset_index().head(50)

,tag,movieId
0,scifi,9141
1,atmospheric,7053
2,action,6783
3,comedy,6368
4,surreal,5584
5,funny,5354
6,based on a book,5194
7,twist ending,4904
8,visually appealing,4691
9,romance,4482


### There also seems to be some further text clean-up required - look at all the variations of 'sci-fi'!

In [217]:
temp = tags[['tag', 'movieId']].groupby('tag').count().sort_values('movieId', ascending=False).reset_index()
temp[temp['tag'].str.lower().str.contains('sci')].head(20)

,tag,movieId
0,scifi,11013
249,science,651
277,bad science,604
543,scientist,329
605,mad scientist,298
749,classic scifi,243
1109,fascism,160
1258,thoughtful scifi,137
1381,good science,121
1533,intelligent scifi,107


In [216]:
def scifi_cleanse(string):
  return str(string).replace('science fiction', 'scifi').replace('sci fi', 'scifi')

tags['tag'] = tags['tag'].apply(scifi_cleanse)

### Here we can see that some of the most popular films can have hundreds of unique tags, while many have just 1

In [218]:
import numpy as np
unique_tags_by_movie = tags.groupby('movieId')['tag'].nunique().reset_index()
unique_tags_by_movie.columns = ['movieId', 'unique_tag_count']
unique_tags_by_movie.sort_values('unique_tag_count', ascending=False, inplace=True)
unique_tags_by_movie.reset_index(inplace=True, drop=True)
unique_tags_by_movie

,movieId,unique_tag_count
0,260,698
1,356,521
2,296,498
3,318,384
4,1103,282
...,...,...
45246,158805,1
45247,176869,1
45248,176867,1
45249,176865,1


### Let's create a frequency table of unique tag counts - we can see that 15% of our films have just 1 tag, while over half have 5 tags or less.

In [219]:
tagcount_freqtable = unique_tags_by_movie.groupby('unique_tag_count').count().reset_index()
tagcount_freqtable.columns = ['unique_tag_count', 'frequency']
tagcount_freqtable['percent'] = tagcount_freqtable['frequency']/(tagcount_freqtable['frequency'].sum())
tagcount_freqtable['cumulative_percent'] = tagcount_freqtable['percent'].cumsum()
display(tagcount_freqtable.head(20))

,unique_tag_count,frequency,percent,cumulative_percent
0,1,7047,0.155731,0.155731
1,2,5379,0.118870,0.274602
2,3,4919,0.108705,0.383306
3,4,3976,0.087865,0.471172
4,5,3228,0.071335,0.542507
5,6,2561,0.056595,0.599103
6,7,2002,0.044242,0.643345
7,8,1694,0.037436,0.680781
8,9,1280,0.028287,0.709067
9,10,1165,0.025745,0.734812


### Let's take a look at that film with 698 tags - id 260

### I think we can all guess what film this is!
### As we can see, there are a lot of meaningless tags ('bite me'), a lot of tags with character names ('yoda' - only really relevant to this particular film/franchise), a lot of tags all meaning pretty much the same thing ('science-fiction', 'sf,science fiction', 'Science Fiction', 'sci-fi') and a lot of free form opinions ('birth of great scifi ideas', 'best movie ever'

### We can also see that there is a lot of upper/lower case versions of similar things

### I think next we should aggregate the other way round, and look at how many tags are used just once/very infrequently

In [220]:
tags[tags['movieId'] == 260].tag.unique()

array(['classic', 'scifi', 'action', 'adventure', 'fantasy',
       'space adventure', 'classic scifi', 'good vs evil', 'aliens',
       'oldie but goodie', 'scifi cult', 'space', 'cult classic',
       'futuristic', 'space action', 'space opera', 'old movie', 'epic',
       'harrison ford', 'space epic', 'action comedy', 'romance',
       'entertaining', 'good story', 'imdb top 250', 'darth vader',
       'luke skywalker', 'the death star', 'religion', 'heros journey',
       'george lucas', 'star wars', 'epic adventure',
       'birth of great scifi ideas', 'old fx quality', 'story driven',
       'bite me',
       'episode what? its cut off, so i dont even know what movie it is',
       'action, scifi', 'quotable', 'exciting', 'fun', 'heroic journey',
       'inventive', 'amazing', 'masterpiece', 'must see', 'universe',
       'sf,scifi', 'incest', 'james earl jones', 'jedi', 'john williams',
       'robots', 'atmospheric', 'future', 'great soundtrack',
       'science fantasy', 'sp

### Here we have the top 20 tags in the dataset, ranked by how many unique films the tag is applied to.
### The top one here is 'bdr' which means 'Blu-ray Disc - Recordable' which is not actually anything to do with the film
### Same logic applies to 'betamax' and 'dvdvideo'
### However some of these tags are indeed useful

In [221]:
movie_count_by_tag = tags.groupby('tag')['movieId'].nunique().reset_index()
movie_count_by_tag.columns = ['tag', 'unique_movie_count']
movie_count_by_tag.sort_values('unique_movie_count', ascending=False, inplace=True)
movie_count_by_tag.reset_index(inplace=True, drop=True)
movie_count_by_tag.head(20)

,tag,unique_movie_count
0,bdr,3948
1,woman director,3493
2,murder,2271
3,independent film,1838
4,comedy,1624
5,nudity topless,1474
6,based on a book,1395
7,clv,1357
8,drama,1307
9,romance,1296


### Let's also construct a frequency table as before:
### An incredible 57.6% of tags only apply to one film!

In [222]:
moviecount_freqtable = movie_count_by_tag.groupby('unique_movie_count').count().reset_index()
moviecount_freqtable.columns = ['unique_movie_count', 'frequency']
moviecount_freqtable['percent'] = moviecount_freqtable['frequency']/(moviecount_freqtable['frequency'].sum())
moviecount_freqtable['cumulative_percent'] = moviecount_freqtable['percent'].cumsum()
display(moviecount_freqtable.head(20))

,unique_movie_count,frequency,percent,cumulative_percent
0,1,37498,0.576059,0.576059
1,2,8345,0.128199,0.704258
2,3,4157,0.063861,0.768120
3,4,2604,0.040004,0.808124
4,5,1795,0.027576,0.835699
5,6,1402,0.021538,0.857237
6,7,1027,0.015777,0.873014
7,8,821,0.012613,0.885627
8,9,633,0.009724,0.895351
9,10,539,0.008280,0.903632


### Creating a list of tags that apply to just one film and scanning through, we can see that the quality of these are not particularly useful

In [223]:
movie_count_by_tag[movie_count_by_tag['unique_movie_count']==1]['tag'].to_list()

['yihad',
 'the virgin suicides',
 'tray',
 'wicked stepmother',
 'travelling players',
 'what the hell did i just watch?',
 'this movie is perfect',
 'travis tope',
 'polemic',
 'the voiceovers here arent as corny as people say',
 'plant food',
 'theological debate',
 'plc hacks',
 'theeffects',
 'psycholgical thriller',
 'theranos',
 'wrongful sentence',
 'transcending abilities',
 'transcendental',
 'pol',
 'the villain from karate kid at the bar',
 'travis oates',
 'yellowstone national park',
 'this movie scares the stuffing out of me!',
 'what are you stuffed with?',
 'playwright:william gibson',
 'plot light',
 'plot point:photo booth',
 'track star',
 'tracking',
 'theology student',
 'this is what they say the notebook was',
 'travis cluff',
 'theone',
 'yegor letov',
 'tracking camera',
 'psychobiddy',
 'transcendentialist',
 'track down',
 'tracking shot',
 'psychobabble',
 'yiddish writer',
 'the views',
 'provoking',
 'yigal amir',
 'this movie is pretty...pretty gay',
 'p

Let's take only the tags that apply to more than 100 films

In [224]:
movie_count_by_tag.head(66)

,tag,unique_movie_count
0,bdr,3948
1,woman director,3493
2,murder,2271
3,independent film,1838
4,comedy,1624
...,...,...
61,teenager,549
62,acting,549
63,history,534
64,cinematography,530
